<a href="https://colab.research.google.com/github/aryansanchiya/Machine-Learning/blob/main/PyTorch_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#PyTorch End to end workflow

*What we are going to cover?*

`1. Data(Prepare and load)`

`2. Build a model`

`3. Fitting the model to Data.`

`4. Making Predictions and evalution on a model(Training)`

`5. Saving and loading Model`

`6. Putting it all together`




In [2]:
import torch
from torch import nn #nn imports all building blocks of neural network
import matplotlib

In [1]:
#Data Preparing and loading
#Data can be anything, Excel SpreadSheet, Images of anykind, Youtube Videos, Audios or Podcast, DNA or Text
#Machine Learning is game of two Parts: 1)Get data into numerical representation 2)Build a modelto learn patterns in that numerical representation.

We will use a linear regression formula to make a straight line with **known** **parameters**.

**Parameters** are something that model learns.

Formula of Linear Regression is :- **y = a + bx**

weight = b

bias = a

In [16]:
#Create known Parameters

weight = 0.8
bias = 0.3

#Create
start = 0
end = 1
step = 0.02

X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias



In [17]:
X[:10],y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3160],
         [0.3320],
         [0.3480],
         [0.3640],
         [0.3800],
         [0.3960],
         [0.4120],
         [0.4280],
         [0.4440]]))

In [14]:
len(X), len(y)

(50, 50)

Tutorial - 43

[Splitting Data into Training and Test sets](https://www.udemy.com/course/pytorch-for-deep-learning/learn/lecture/32686050#content)

**One of the most important concept in machine learning**

Lets create training and test set for the data.

In [19]:
train_split = int(0.8 * len(X))
X_train,y_train = X[ :train_split],y[ :train_split]
X_test,y_test = X[train_split: ],y[train_split: ]

len(X_train),len(y_train),len(X_test),len(y_train)

(40, 40, 10, 40)